# Instalando pacotes


In [ ]:
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support
  python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support python3.7
  python3.7-minimal
0 upgraded, 6 newly installed, 0 to remove and 32 not upgraded.
Need to get 4,698 kB of archives.
After this operation, 17.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-minimal amd64 3.7.17-1+jammy1 [608 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-minima

In [ ]:
!sudo apt-get install python3.7-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3.7-lib2to3
The following NEW packages will be installed:
  python3.7-distutils python3.7-lib2to3
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 313 kB of archives.
After this operation, 1,229 kB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-lib2to3 all 3.7.17-1+jammy1 [124 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-distutils all 3.7.17-1+jammy1 [189 kB]
Fetched 313 kB in 1s (296 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize fronte

In [ ]:
!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 50.2 MB/s eta 0:00:00


# Código

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS')

In [ ]:
%cd /content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS

/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS


In [ ]:
import os
import sys
import time
import glob
import random
import numpy as np
import torch
import utils
import logging
import argparse
import torch.nn as nn
import genotypes
import torch.utils
import torchvision.datasets as dset
import torch.backends.cudnn as cudnn
import uuid
from model import NetworkCIFAR as Network


In [ ]:
parser = argparse.ArgumentParser("cifar")
parser.add_argument('--data', type=str, default='./dataset/', help='location of the data corpus')
parser.add_argument('--dataset', type=str, default='cifar10', help='dataset')
parser.add_argument('--batch_size', type=int, default=96, help='batch size')
parser.add_argument('--learning_rate', type=float, default=0.025, help='init learning rate')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
parser.add_argument('--weight_decay', type=float, default=3e-4, help='weight decay')
parser.add_argument('--report_freq', type=float, default=50, help='report frequency')
parser.add_argument('--gpu_id', type=int, default=0, help='gpu device id')
parser.add_argument('--epochs', type=int, default=50, help='num of training epochs')
parser.add_argument('--init_channels', type=int, default=36, help='num of init channels')
parser.add_argument('--layers', type=int, default=20, help='total number of layers')
parser.add_argument('--model_path', type=str, default='saved_models', help='path to save the model')
parser.add_argument('--auxiliary', action='store_true', default=False, help='use auxiliary tower')
parser.add_argument('--auxiliary_weight', type=float, default=0.4, help='weight for auxiliary loss')
parser.add_argument('--cutout', action='store_true', default=False, help='use cutout')
parser.add_argument('--cutout_length', type=int, default=16, help='cutout length')
parser.add_argument('--drop_path_prob', type=float, default=0.2, help='drop path probability')
parser.add_argument('--save', type=str, default='EXP', help='experiment name')
parser.add_argument('--seed', type=int, default=0, help='random seed')
parser.add_argument('--arch', type=str, default='du_darts_c10_s0', help='which architecture to use')
parser.add_argument('--grad_clip', type=float, default=5, help='gradient clipping')
parser.add_argument('--pretrained_ckpt', type=str, default=None, help='path to pretrained checkpoint')
args = parser.parse_args(args=['--dataset', 'brainMRI', '--arch', 'du_darts_c10_s0', '--pretrained_ckpt', '/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS/log/search-brainMRI--20240206-144420/weights.pt'])


In [ ]:
def main():
    if not torch.cuda.is_available():
        logging.info('no gpu device available')
        sys.exit(1)

    utils.set_seed(seed=args.seed)

    logging.info('gpu device = %d' % args.gpu_id)
    logging.info("args = %s", args)
    print(f'gpu device = {args.gpu_id}')
    print(f'args = {args}')

    genotype = eval("genotypes.%s" % args.arch)
    # model = Network(args.init_channels, CIFAR_CLASSES, args.layers, args.auxiliary, genotype)
    model = torch.load(args.pretrained_ckpt)
    model = model.cuda()

    params = utils.count_parameters_in_MB(model)
    logging.info("param size = %fMB", params)
    print(f'param size = {params}MB')

    criterion = nn.CrossEntropyLoss()
    criterion = criterion.cuda()
    optimizer = torch.optim.SGD(
        model.parameters(),
        args.learning_rate,
        momentum=args.momentum,
        weight_decay=args.weight_decay
    )

    if args.dataset == 'cifar10':
        train_transform, valid_transform = utils.data_transforms_cifar10(args)
        train_data = dset.CIFAR10(root=args.data+args.dataset, train=True, download=True, transform=train_transform)
        valid_data = dset.CIFAR10(root=args.data+args.dataset, train=False, download=True, transform=valid_transform)
    elif args.dataset == 'cifar100':
        train_transform, valid_transform = utils.data_transforms_cifar100(args)
        train_data = dset.CIFAR100(root=args.data+args.dataset, train=True, download=True, transform=train_transform)
        valid_data = dset.CIFAR100(root=args.data+args.dataset, train=False, download=True, transform=valid_transform)
    elif args.dataset == 'brainMRI':
        ext_list = ['.jpg', '.JPG', '.jpeg', '.png']
        root_brain = '/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS/dataset/brain_tumor_dataset'

        train_transform, valid_transform = utils.data_transforms_brainMRI(args)
        train_data = dset.DatasetFolder(root = root_brain, loader =  utils.default_loader , extensions= ext_list, transform= train_transform)
        valid_data = dset.DatasetFolder(root = root_brain, loader =  utils.default_loader , extensions= ext_list, transform= valid_transform)
    else:

        raise ValueError('No Defined Dataset!!!')

    train_queue = torch.utils.data.DataLoader(
        train_data, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=2)

    valid_queue = torch.utils.data.DataLoader(
        valid_data, batch_size=args.batch_size, shuffle=False, pin_memory=True, num_workers=2)

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, float(args.epochs))

    best_val_acc = 0.
    for epoch in range(args.epochs):
        scheduler.step()
        print(f'\n\nepoch {epoch} lr {scheduler.get_lr()[0]}')
        logging.info('epoch %d lr %e', epoch, scheduler.get_lr()[0])
        model.drop_path_prob = args.drop_path_prob * epoch / args.epochs

        train_acc, train_obj = train(train_queue, model, criterion, optimizer)
        print(f'train_acc {train_acc}')
        logging.info('train_acc %f', train_acc)

        with torch.no_grad():
            valid_acc, valid_obj = infer(valid_queue, model, criterion)
            if valid_acc > best_val_acc:
                best_val_acc = valid_acc
                utils.save_entire_model(model, os.path.join(args.save, 'best_weights.pt'))
            logging.info('valid_acc %f\tbest_val_acc %f', valid_acc, best_val_acc)
            print(f'valid_acc {valid_acc} best_val_acc {best_val_acc}')

        utils.save_entire_model(model, os.path.join(args.save, 'weights.pt'))
    # with open('./results/retrain_%s.txt' % args.dataset, 'a') as f:
    #     f.write("model:%s\tparams:%.2f\tbest_val_acc: %s\n"
    #             % (args.arch, params, best_val_acc))

    return best_val_acc


def train(train_queue, model, criterion, optimizer):
    objs = utils.AverageMeter()
    top1 = utils.AverageMeter()
    top5 = utils.AverageMeter()
    model.train()

    for step, (input, target) in enumerate(train_queue):
        # input = Variable(input).cuda()
        # target = Variable(target).cuda(async=True)
        input = input.cuda()
        target = target.cuda(non_blocking=True)

        optimizer.zero_grad()
        # print(model(input))
        # logits, logits_aux = model(input)
        logits = model(input)
        # loss = criterion(logits_aux, target)
        loss = criterion(logits, target)
        if args.auxiliary:
            loss_aux = criterion(logits, target)
            loss += args.auxiliary_weight * loss_aux
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
        optimizer.step()

        prec1, prec5 = utils.accuracy(logits, target, topk=(1, 2))
        n = input.size(0)
        objs.update(loss.item(), n)
        top1.update(prec1.item(), n)
        top5.update(prec5.item(), n)

        if step % args.report_freq == 0:
            logging.info('train %03d %e %f %f', step, objs.avg, top1.avg, top5.avg)
            print(f'train {step} {objs.avg} {top1.avg} {top5.avg}')
    return top1.avg, objs.avg


def infer(valid_queue, model, criterion):
    objs = utils.AverageMeter()
    top1 = utils.AverageMeter()
    top5 = utils.AverageMeter()
    model.eval()

    for step, (input, target) in enumerate(valid_queue):
        # input = Variable(input).cuda()
        # target = Variable(target).cuda(async=True)
        input = input.cuda()
        target = target.cuda(non_blocking=True)

        logits = model(input)
        loss = criterion(logits, target)

        prec1, prec5 = utils.accuracy(logits, target, topk=(1, 2))
        n = input.size(0)
        objs.update(loss.item(), n)
        top1.update(prec1.item(), n)
        top5.update(prec5.item(), n)

        if step % args.report_freq == 0:
            logging.info('valid %03d %e %f %f', step, objs.avg, top1.avg, top5.avg)
            print(f'valid {step} {objs.avg} {top1.avg} {top5.avg}')
    return top1.avg, objs.avg



In [ ]:
# args.save = args.arch
args.save = 'log/{}-{}-{}'.format(args.save, time.strftime("%Y%m%d-%H%M%S"), str(uuid.uuid4()))
utils.create_exp_dir(args.save, scripts_to_save=glob.glob('*.py'))

log_format = '%(asctime)s %(message)s'
logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format=log_format, datefmt='%m/%d %I:%M:%S %p')
fh = logging.FileHandler(os.path.join(args.save, 'log.txt'))
fh.setFormatter(logging.Formatter(log_format))
logging.getLogger().addHandler(fh)

if args.dataset == 'cifar10':
    CIFAR_CLASSES = 10
elif args.dataset == 'cifar100':
    CIFAR_CLASSES = 100
elif args.dataset == 'brainMRI':
    CIFAR_CLASSES = 2
else:
    raise ValueError('No Defined Dataset!!!')


if __name__ == '__main__':
    utils.run_func(args, main)
